In [6]:
# %load effort.py
import numpy as np
from keras.preprocessing import image

def load_seq(filename, B, N, F, D):
    with open(filename) as file:
        text = file.read()
        text = text.splitlines()
    dat = np.zeros((len(text), 10))
    for i in range(0,len(text)):
        l = text[i].split(',')
        for j in range(0,len(l)):
            dat[i][j] = float(l[j])
    
    data = np.full((B, N, F, D), np.nan)
    label = np.full((B, N, F), np.nan, dtype=int)
    num_targets = np.zeros((B, F), dtype=int)
    i = 0
    k = 0
    while dat[i][0] <= B * F:
        b = int((dat[i][0]-1) // F)
        r = int((dat[i][0]-1) % F)
        data[b][k][r][:] = dat[i][2:6]
        label[b][k][r] = int(dat[i][1])
        i += 1
        k += 1
        while k >= N and dat[i][0] == dat[i-1][0]:
            i += 1
        if i > 0 and dat[i][0] != dat[i-1][0]:
            num_targets[b][r] = k
            k = 0
    return (data, label)
    
def loadTable(filename, B, M, N, F, D):
    (detections, dummy) = load_seq(filename + "det/det.txt", B, N, F, D)
    img = image.load_img(filename + "det/000001-acf.jpg")
    sz = img.size
    detections[0,:,:,0] = detections[0,:,:,0]/sz[0] - 0.5
    detections[0,:,:,1] = detections[0,:,:,1]/sz[1] - 0.5
    detections[0,:,:,2] = detections[0,:,:,2]/sz[0] - 0.5
    detections[0,:,:,3] = detections[0,:,:,3]/sz[1] - 0.5
    (tracks, labels) = load_seq(filename + "gt/gt.txt", B, M, F, D)
    tracks[0,:,:,0] = tracks[0,:,:,0]/sz[0] - 0.5
    tracks[0,:,:,1] = tracks[0,:,:,1]/sz[1] - 0.5
    tracks[0,:,:,2] = tracks[0,:,:,2]/sz[0] - 0.5
    tracks[0,:,:,3] = tracks[0,:,:,3]/sz[1] - 0.5
    return (tracks, detections, labels)

# Example: traindata = loadTable("/home/raj/Documents/project/amilan-rnntracking-64d477848af3/src/2DMOT2015Labels/train/ADL-Rundle-8/", 1, 20, 20, 20, 4)

import os
import PIL
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Input, Flatten, Lambda, SimpleRNN, LSTM
from keras import backend as K
from keras.models import Model

def getDA(N, M, hidden_units):
    a = Input(shape=(N,M))
    b = LSTM(hidden_units, return_sequences=True, activation='tanh')(a)
    c = Dense(M, activation='softmax')(b)
    model = Model(input=a, output=c)
    return model

def getDistMat(tracks, detections, F, N, M):
    label = np.zeros((F, N, M))
    Ct = np.full((F, N, M), 4, dtype=float)
    for t in range(0,F):
        for i in range(0,N):
            for j in range(0,M):
                dist = np.linalg.norm(tracks[0,i,t,:]-detections[0,j,t,:])
                if not np.isnan(dist) and not np.isinf(dist):
                    Ct[t, i, j] = dist
            ind = np.argmin(Ct[t,i,:])
            label[t,i,ind] = 1
    return (Ct, label)

N = 2
M = 20
F = 500
B = 1
D = 4
DA = getDA(N, M, 100)

(train_tracks, train_det, dummy) = loadTable("/home/raj/Documents/project/amilan-rnntracking-64d477848af3/src/2DMOT2015Labels/train/ADL-Rundle-8/", B, N, M, F, D)

(test_tracks, test_det, dummy) = loadTable("/home/raj/Documents/project/amilan-rnntracking-64d477848af3/src/2DMOT2015Labels/train/ADL-Rundle-6/", B, N, M, F, D)

(train_x, train_y) = getDistMat(train_tracks, train_det, F, N, M)
(test_x, test_y) = getDistMat(test_tracks, test_det, int(F/5), N, M)

DA.compile( loss="categorical_crossentropy", optimizer="Nadam", metrics = ['categorical_accuracy'])
#history1 = DA.fit(train_x, train_y, batch_size=5, nb_epoch=30, validation_data=(test_x, test_y))


In [7]:
history1 = DA.fit(train_x, train_y, batch_size=5, nb_epoch=50, validation_data=(test_x, test_y))

Train on 500 samples, validate on 100 samples
Epoch 1/50
500/500 [==============================] - 1s - loss: 2.2875 - categorical_accuracy: 0.2550 - val_loss: 1.8535 - val_categorical_accuracy: 0.3600

In [8]:
pred = DA.predict(test_x)
pred.shape

(100, 2, 20)

In [12]:
da1 = np.argmax(pred, axis=2)
true_det = np.zeros((int(F/5), D), dtype=float)
gt = np.zeros((int(F/5), D), dtype=float)
error = np.zeros((int(F/5),1), dtype=float)
for i in range(0,int(F/5)):
    true_det[i,:] = np.multiply(test_det[0,da1[i,0],i,:]+0.5, np.array((1920, 1080, 1920, 1080)))
    gt[i,:] = np.multiply(test_tracks[0,0,i,:]+0.5, np.array((1920, 1080, 1920, 1080)))
    error[i] = np.linalg.norm(gt[i,:] - true_det[i,:])

In [13]:
true_det2 = np.zeros((int(F/5), D), dtype=float)
gt2 = np.zeros((int(F/5), D), dtype=float)
error2 = np.zeros((int(F/5),1), dtype=float)
for i in range(0,int(F/5)):
    true_det2[i,:] = np.multiply(test_det[0,da1[i,1],i,:]+0.5, np.array((1920, 1080, 1920, 1080)))
    gt2[i,:] = np.multiply(test_tracks[0,1,i,:]+0.5, np.array((1920, 1080, 1920, 1080)))
    error2[i] = np.linalg.norm(gt2[i,:] - true_det2[i,:])

In [14]:
gt

array([[ 1703.,   385.,   157.,   339.],
       [ 1699.,   383.,   159.,   341.],
       [ 1697.,   383.,   159.,   343.],
       [ 1695.,   383.,   159.,   343.],
       [ 1693.,   381.,   159.,   347.],
       [ 1689.,   381.,   161.,   349.],
       [ 1687.,   381.,   161.,   349.],
       [ 1685.,   379.,   161.,   353.],
       [ 1683.,   379.,   161.,   353.],
       [ 1679.,   379.,   163.,   355.],
       [ 1677.,   379.,   165.,   357.],
       [ 1675.,   377.,   165.,   359.],
       [ 1673.,   377.,   165.,   361.],
       [ 1669.,   377.,   167.,   361.],
       [ 1667.,   375.,   167.,   365.],
       [ 1665.,   375.,   167.,   367.],
       [ 1663.,   375.,   167.,   367.],
       [ 1659.,   375.,   169.,   369.],
       [ 1657.,   373.,   169.,   371.],
       [ 1655.,   373.,   169.,   373.],
       [ 1653.,   373.,   171.,   375.],
       [ 1649.,   371.,   173.,   377.],
       [ 1647.,   371.,   173.,   379.],
       [ 1645.,   371.,   173.,   379.],
       [ 1643., 

In [15]:
true_det

array([[ 1689.  ,   385.  ,   146.62,   332.71],
       [ 1689.  ,   385.  ,   146.62,   332.71],
       [ 1664.  ,   418.  ,   135.42,   307.29],
       [ 1668.  ,   371.  ,   159.84,   362.7 ],
       [ 1692.  ,   371.  ,   159.84,   362.7 ],
       [ 1666.  ,   385.  ,   146.62,   332.71],
       [ 1668.  ,   371.  ,   159.84,   362.7 ],
       [ 1668.  ,   371.  ,   159.84,   362.7 ],
       [ 1727.  ,   418.  ,   135.42,   307.29],
       [ 1689.  ,   385.  ,   146.62,   332.71],
       [ 1689.  ,   385.  ,   146.62,   332.71],
       [ 1689.  ,   408.  ,   146.62,   332.71],
       [ 1689.  ,   385.  ,   146.62,   332.71],
       [ 1668.  ,   371.  ,   159.84,   362.7 ],
       [ 1668.  ,   371.  ,   159.84,   362.7 ],
       [ 1668.  ,   371.  ,   159.84,   362.7 ],
       [ 1666.  ,   385.  ,   146.62,   332.71],
       [ 1594.  ,   396.  ,   159.84,   362.7 ],
       [ 1594.  ,   396.  ,   159.84,   362.7 ],
       [ 1643.  ,   371.  ,   159.84,   362.7 ],
       [ 1594.  ,   

In [17]:
gt2

array([[ 1293.,   455.,    83.,   213.],
       [ 1293.,   455.,    83.,   213.],
       [ 1293.,   455.,    83.,   213.],
       [ 1293.,   455.,    83.,   213.],
       [ 1295.,   455.,    83.,   213.],
       [ 1297.,   455.,    83.,   213.],
       [ 1299.,   455.,    83.,   213.],
       [ 1301.,   455.,    83.,   213.],
       [ 1303.,   455.,    83.,   213.],
       [ 1305.,   455.,    83.,   213.],
       [ 1307.,   455.,    83.,   213.],
       [ 1309.,   455.,    83.,   213.],
       [ 1313.,   455.,    83.,   215.],
       [ 1315.,   455.,    83.,   215.],
       [ 1317.,   455.,    83.,   215.],
       [ 1319.,   455.,    83.,   215.],
       [ 1321.,   455.,    83.,   215.],
       [ 1323.,   455.,    83.,   215.],
       [ 1325.,   455.,    83.,   215.],
       [ 1327.,   455.,    83.,   215.],
       [ 1331.,   457.,    83.,   215.],
       [    7.,   237.,   241.,   673.],
       [    5.,   237.,   249.,   673.],
       [    5.,   237.,   255.,   673.],
       [    5., 

In [18]:
true_det2

array([[ 1800.   ,   483.   ,    94.66 ,   214.81 ],
       [ 1312.   ,   503.   ,    61.514,   139.59 ],
       [ 1234.   ,   415.   ,   103.17 ,   234.13 ],
       [ 1668.   ,   371.   ,   159.84 ,   362.7  ],
       [ 1318.   ,   537.   ,    52.   ,   118.   ],
       [ 1741.   ,   473.   ,   113.37 ,   257.27 ],
       [ 1668.   ,   371.   ,   159.84 ,   362.7  ],
       [  257.   ,   455.   ,   113.37 ,   257.27 ],
       [ 1274.   ,   551.   ,    43.624,    98.993],
       [ 1278.   ,   545.   ,    47.794,   108.46 ],
       [ 1689.   ,   385.   ,   146.62 ,   332.71 ],
       [ 1619.   ,   396.   ,   159.84 ,   362.7  ],
       [ 1286.   ,   545.   ,    47.794,   108.46 ],
       [ 1619.   ,   396.   ,   159.84 ,   362.7  ],
       [ 1619.   ,   396.   ,   159.84 ,   362.7  ],
       [ 1619.   ,   396.   ,   159.84 ,   362.7  ],
       [ 1336.   ,   500.   ,    67.474,   153.11 ],
       [ 1295.   ,   461.   ,    87.838,   199.32 ],
       [ 1291.   ,   454.   ,    94.66 ,   214

In [21]:
sum = 0
for i in error2:
    if i>50:
        sum += 1
sum

83

In [23]:
vars(history1)

{'epoch': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49],
 'history': {'categorical_accuracy': [0.25500000491738317,
   0.28900000616908073,
   0.29900000527501108,
   0.35600000761449335,
   0.38600000657141209,
   0.41900000639259816,
   0.44800000570714471,
   0.45500000603497026,
   0.46100000649690626,
   0.48600000597536563,
   0.51500000700354576,
   0.54000000670552251,
   0.55000000417232509,
   0.55800000749528411,
   0.59000000581145284,
   0.59900000251829622,
   0.60800000376999375,
   0.60900000348687167,
   0.63500000536441803,
   0.63200000233948228,
   0.65200000226497645,
   0.6590000027418137,
   0.68500000208616252,
   0.68300000265240668,
   0.70200000256299977,
   0.71300000309944156,
   0.73900000154972079,
   0.7379999984800